In [26]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'PHONE',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
path = r'C:\Users\fajar\Documents\Python\Data\karma beach.xlsx'
# zoho_df = pd.read_csv(path, dtype='string')
zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
0,zcrm_3584690000007666275,2019-04-02 06:27:44,2017-06-30 00:00:00,Pooja,Vaswani,poojakanal@gmail.com,971585680048,empty,United Arab Emirates,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
1,zcrm_3584690000007840074,2019-04-02 06:46:04,2017-07-04 00:00:00,Oreste,Lentidoro,lentidoro@gmail.com,61410699788,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
2,zcrm_3584690000007849782,2019-04-02 06:47:07,2017-07-04 00:00:00,Shierley,Koval,shierleykoval@gmail.com,622153688631,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
3,zcrm_3584690000007896028,2019-04-02 06:52:18,2017-06-30 00:00:00,Wesley,Powell,wez.powell@gmail.com,61459554763,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
4,zcrm_3584690000007931926,2019-04-02 06:56:31,2015-03-11 00:00:00,Maidy,Kusmana,maivaleria@gmail.com,62811921180,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17635,empty,empty,2024-06-27 13:47:35,Rob,Maconachie,rob@channelmobile.co.za,270836538441,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17636,empty,empty,2024-07-02 12:37:05,Beatrice,Graubart,bzgraubart@gmail.com,18326710560,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17637,empty,empty,2025-02-12 14:48:44,Benny,Risanto,benny.risanto@karmagroup.com,62811392201,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17638,empty,empty,2024-04-02 18:01:36,Adhitya,Karmagroup,adhitya.putra+1@karmagroup.com,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [28]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [29]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

'Email'

In [30]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [31]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
849,zcrm_3584690000206499154,2020-07-22 08:24:35,empty,Thierry,Nolibr,thierry1970@yahoo.com,6282147470330,Bali,Indonesia,Karma Resorts,Karma Beach,Web Enquiry,Karma Beach Bali offer sign up,empty,empty
964,zcrm_3584690000243851842,2020-11-09 11:38:47,empty,Qaul,Siregar,qaulsiregar@gmail.com,6282166526245,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty
993,zcrm_3584690000243878881,2020-11-09 12:44:32,empty,Simone,Tait,simotait@gmail.com,628118306525,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty
1005,zcrm_3584690000243888528,2020-11-09 13:43:10,empty,Salvatore,Adaldo,salvatoreadaldosg@gmail.com,6594499542,empty,Singapore,Karma Resorts,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty
1055,zcrm_3584690000243901901,2020-11-09 15:50:44,empty,Ari Widya,Merta,are.rick@ymail.com,6282147034957,Bali,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP,Karma Beach VIP membership sign up,General,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17628,empty,empty,2024-06-25 15:43:32,Andrew,Mills,seniormills1@gmail.com,447966762289,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17629,empty,empty,2024-06-25 15:48:11,Mrs Monica,Chung,monicajchung@gmail.com,6591540709,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17630,empty,empty,2024-06-25 15:49:38,Lamzira,Yassine,yassinelamzira@yahoo.fr,330769632219,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
17633,empty,empty,2024-06-27 13:46:16,Aprilia cindy,Damian,c1ndy_15@hotmail.com,6583465600,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [32]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
0,zcrm_3584690000007666275,2019-04-02 06:27:44,2017-06-30 00:00:00,Pooja,Vaswani,poojakanal@gmail.com,971585680048,empty,United Arab Emirates,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
1,zcrm_3584690000007840074,2019-04-02 06:46:04,2017-07-04 00:00:00,Oreste,Lentidoro,lentidoro@gmail.com,61410699788,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
2,zcrm_3584690000007849782,2019-04-02 06:47:07,2017-07-04 00:00:00,Shierley,Koval,shierleykoval@gmail.com,622153688631,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
3,zcrm_3584690000007896028,2019-04-02 06:52:18,2017-06-30 00:00:00,Wesley,Powell,wez.powell@gmail.com,61459554763,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
4,zcrm_3584690000007931926,2019-04-02 06:56:31,2015-03-11 00:00:00,Maidy,Kusmana,maivaleria@gmail.com,62811921180,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16980,empty,empty,2024-06-27 13:30:44,Pauline,Huon,pauline.huon@hotmail.fr,330769632219,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16981,empty,empty,2024-07-02 12:37:05,Beatrice,Graubart,bzgraubart@gmail.com,18326710560,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16982,empty,empty,2025-02-12 14:48:44,Benny,Risanto,benny.risanto@karmagroup.com,62811392201,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16983,empty,empty,2024-04-02 18:01:36,Adhitya,Karmagroup,adhitya.putra+1@karmagroup.com,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [33]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
16957,empty,empty,2024-02-17 18:16:19,IT,Test,adi.sumardiyasa@karmagroup.com,823218021654,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [34]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [35]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [36]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [37]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
6767,zcrm_3584690000491845127,2024-04-14 00:26:53,empty,Ankit,Agarwal,ankit@yotours.in,917276282020,empty,India,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
6768,zcrm_3584690000491845156,2024-04-14 00:29:07,empty,Priyanka,Mangal,info@yotours.in,918451823654,empty,India,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
8664,zcrm_3584690000512153086,2024-10-17 13:15:08,empty,Tamara,Skroman,supermarkettours@mail.ru,6281225297099,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
16526,empty,empty,2020-07-30 11:50:05.519000,Aaron,-,aaron@agoda.com,empty,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16543,empty,empty,2020-10-24 12:03:03.613000,Sandra,-,sandra.tjoa@agoda.com,08123827678,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16567,empty,empty,2020-12-20 17:33:03.566000,Ronal,-,balitourgemilangintown@gmail.con,081236795101,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty


In [38]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
0,zcrm_3584690000007666275,2019-04-02 06:27:44,2017-06-30 00:00:00,Pooja,Vaswani,poojakanal@gmail.com,971585680048,empty,United Arab Emirates,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
1,zcrm_3584690000007840074,2019-04-02 06:46:04,2017-07-04 00:00:00,Oreste,Lentidoro,lentidoro@gmail.com,61410699788,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
2,zcrm_3584690000007849782,2019-04-02 06:47:07,2017-07-04 00:00:00,Shierley,Koval,shierleykoval@gmail.com,622153688631,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
3,zcrm_3584690000007896028,2019-04-02 06:52:18,2017-06-30 00:00:00,Wesley,Powell,wez.powell@gmail.com,61459554763,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
4,zcrm_3584690000007931926,2019-04-02 06:56:31,2015-03-11 00:00:00,Maidy,Kusmana,maivaleria@gmail.com,62811921180,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16973,empty,empty,2024-06-27 13:30:44,Pauline,Huon,pauline.huon@hotmail.fr,330769632219,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16974,empty,empty,2024-07-02 12:37:05,Beatrice,Graubart,bzgraubart@gmail.com,18326710560,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16975,empty,empty,2025-02-12 14:48:44,Benny,Risanto,benny.risanto@karmagroup.com,62811392201,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16976,empty,empty,2024-04-02 18:01:36,Adhitya,Karmagroup,adhitya.putra+1@karmagroup.com,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [39]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
5030,zcrm_3584690000472907352,2023-12-14 08:24:49,empty,Shaun,Duenhofen,shaun.duenhofen@karmakandara.com,6281138006892,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
5031,zcrm_3584690000472907381,2023-12-14 09:12:09,empty,James,Casey,james.casey@karmagroup.com,628123601872,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
5406,zcrm_3584690000476213047,2024-01-10 06:59:19,empty,Michael,Wood,michael.wood@karmagroup.com,628111111111,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
5453,zcrm_3584690000479790119,2024-01-13 11:07:28,2023-12-09 00:00:00,Agus,Maryana,agus.maryana@karmagroup.com,628122730101,empty,empty,Karma Resorts,Karma Beach,Other,Karma Beach Bali Walk-in Guest,Karma Beach,empty
9764,zcrm_3584690000522488067,2025-01-27 10:24:16,empty,Arditra,Test Booking,resmgr@karmaresorts.com,623618482200,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
16508,empty,empty,2020-06-20 22:44:08.239000,Agnes,-,agness.houten@karmagroup.com,0812369966901,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16511,empty,empty,2020-06-21 17:49:35.965000,Andy,-,andrew.easther@karmagroup.com,087846236917,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16517,empty,empty,2020-06-28 14:46:06.192000,Mona,Bhartolomay,mona.bhartolomay@karmaestates.com,08113863499,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16587,empty,empty,2021-02-02 15:20:13.670000,Diego,-,budiman.sujatmika@karmagroup.com,empty,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty
16707,empty,empty,2021-08-17 14:35:06.551000,Eleana,-,eleana.kealy@karmagroup.com,447581698196,empty,Indonesia,Karma Resorts,Other,COVID Register,COVID Register,Karma Beach,empty


In [40]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Mobile
0,zcrm_3584690000007666275,2019-04-02 06:27:44,2017-06-30 00:00:00,Pooja,Vaswani,poojakanal@gmail.com,971585680048,empty,United Arab Emirates,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
1,zcrm_3584690000007840074,2019-04-02 06:46:04,2017-07-04 00:00:00,Oreste,Lentidoro,lentidoro@gmail.com,61410699788,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
2,zcrm_3584690000007849782,2019-04-02 06:47:07,2017-07-04 00:00:00,Shierley,Koval,shierleykoval@gmail.com,622153688631,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
3,zcrm_3584690000007896028,2019-04-02 06:52:18,2017-06-30 00:00:00,Wesley,Powell,wez.powell@gmail.com,61459554763,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
4,zcrm_3584690000007931926,2019-04-02 06:56:31,2015-03-11 00:00:00,Maidy,Kusmana,maivaleria@gmail.com,62811921180,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16960,empty,empty,2024-06-23 18:20:00,Georgia,empty,calingeorgiana76@gmail.com,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16961,empty,empty,2024-06-26 16:31:34,Veronica,empty,empty,79532098939,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16962,empty,empty,2024-06-27 13:30:44,Pauline,Huon,pauline.huon@hotmail.fr,330769632219,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty
16963,empty,empty,2024-07-02 12:37:05,Beatrice,Graubart,bzgraubart@gmail.com,18326710560,empty,empty,Karma Resorts,Karma Beach,Karma Beach Walk-In,Karma Beach Bali Walk-in Guest,Karma Beach,empty


In [41]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [42]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [43]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [44]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

676

In [45]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 17640
Jumlah Format Email Salah : 1
Jumlah Setelah Clean Format Email Salah : 17639
Jumlah Duplicate (Phone, Mobile dan Email) : 654
Jumlah Setelah Clean Duplicate : 16985
Jumlah Lead Test : 1
Jumlah Setelah Clean Lead Test : 16984
Jumlah Email OTA : 6
Jumlah Setelah Clean Email OTA : 16978
Jumlah Email Karma : 13
Jumlah Setelah Clean Email Karma : 16965
Jumlah Wash Member : 1059
Jumlah Wash Booking : 128
Jumlah Setelah Wash Member : 15778
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 1
Jumlah Setelah Wash Phone Email Blank : 15777
Contact Type
Jumlah Email Only : 1269
Jumlah Phone Only : 77
Jumlah Email dan Phone : 14431


In [46]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 17640
Jumlah Wash Dedup : 676
Jumlah Wash Member : 1059
Jumlah Wash Booking : 128
Jumlah Data Akhir : 15777


In [47]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar\Documents\Python\Data\karma beach cleand.csv', index=False)
zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\karma beach cleand.xlsx', index=False)